**Best Location for a New Mexican Restaurant**

*Import libraries, set-up notebook*

In [ ]:
import numpy as np                                                       # library to handle data in a vectorized manner
from numpy.random import randint 

import pandas as pd                                                      # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json                                                              # library to handle JSON files
from pandas.io.json import json_normalize
import requests                                                          # library to handle requests

from geopy.geocoders import Nominatim                                    # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="foursquare_agent")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                          # display requested dataframes in pandas format

*Load Michigan city lat, lng, pop and growth rate data into a dataframe*

In [2]:
CLIENT_ID = 'ZQKKMGNITTOGCMFEPVIQH5W1MZRDABZEX52IROB0KLFW1LVI'
CLIENT_SECRET = '2KOHRJGACJXUAK3BIRM1P0CORACXOJ15GPUJZQNHBNBH3MMP'
#CLIENT_ID = 'MJ1EJQRTUUE3ZGU4NE2DV2P2HZWO0U5E3QU1EFLOT51GQQC3'           # your Foursquare ID
#CLIENT_SECRET = 'UFY2KX53GD0PRWMZQ0T4BU2ANOHX24RZM2HGXNHAWODLRGL1'       # your Foursquare Secret
print('Your credentials are saved.\n')

VERSION = '20201015'
LIMIT = 100
radius = 15000
search_query = 'Mexican'
pluralNames = "Mexican%20Restaurants"

capMex = pd.DataFrame(columns=['id','name','city'])  # dataFrame reserved for function

print('Your fixed variables are saved.\n')

cities = pd.read_csv(r'C:\Users\Jim_2\Downloads\simplemaps\uscities.csv') # location of https://simplemaps.com/data/us-cities download
cities = cities.reset_index(drop=True)                                    # add index back onto data
cities = cities.loc[(cities['city'].isin(['Detroit','Kalamazoo',\
    'Lansing','Grand Rapids'])) & (cities['state_id'] =="MI")]            # locate data for Michigan cities
cities2 = cities[['city','lat','lng','population']].copy()                # select required columns from dataframe
growth_lst = [-.006,.0301,.038,.0917]                                        # googled growth rate value since last census
cities2['growth_rate'] = growth_lst                                       # add growth rate to dataframe                        
cities2.index = np.arange(1, len(cities2)+ 1 )                            # resets index to 1

print("\nWe will be searching in the following Michigan cities for Mexican restaurant data!\n")
cities3 = cities2.values                                                  # save array value for later use
cities2 
print('\n')

for city, row in cities2.iterrows():                                      # loop through cities to gather foursquare data
    lat = row['lat']                                                      # set latitude variable for foursquare function
    lng = row['lng']                                                      # set longitude variable for foursquare function
    city = row['city']                                                    # set city variable for foursquare function
    
   # it returns a dataframe that provides a list of Mexican restaurants in specific metro areas, their id, categories, address, lat and lng
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&pluralNames={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT, pluralNames)
    
    global onlyDT, onlyGR, onlyLS, onlyKZ, mexOnly                      # set global variables inside function
    global capMex                                                       # dataFrame reserved for function
    subs = "Mexican Restaurant"                                         # set search variable
       
    results = requests.get(url).json()                                  # make the GET request
    mexican = results['response']['venues']                             # grab data
    mex_df = pd.json_normalize(mexican)                                 # tranform venues into a dataframe
    
    mexOnly= mex_df[['id','name','categories','location.address']].copy() # select required Foursquare data
    mex2 = mexOnly['categories'].copy()                                 # get object column from mex2
    mex2 = pd.DataFrame(mex2).astype(str)                               # change to string datatype
    mex2["value"]= mex2['categories'].str.find(subs)                    # add bool value column with identify "Mexican Restaurant"
    
    mexOnly['value'] = mex2['value']                                    # add value column to mexOnly
    mexOnly = mexOnly[mexOnly.value != -1]                              # remove row values that are equal to -1 (not Mex restaurants)
    mexOnly = mexOnly[['id','name']]                                    # provides list of only Mexican restaurants in city
    mexOnly = mexOnly.reset_index(drop=True)                            # resets row index
    mexOnly['city'] = city                                              # add city name to dataframe 
    
    # saving output data while looping through Michigan cities
    if city == "Detroit":
        onlyDT = mexOnly
        onlyDT.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\onlyDT.csv', index=False, header=True) # saving file to disk
        capMex = mexOnly                                                # save total output of Detroit data
        dtCount = len(onlyDT.index)
        
    elif city == "Grand Rapids":
        onlyGR = mexOnly
        onlyGR.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\onlyGR.csv', index=False, header=True) # saving file to disk
        capMex = capMex.append(onlyGR)                                  # append total output of Grand Rapids data
        grCount = len(onlyGR.index)
        
    elif city == "Lansing":
        onlyLS = mexOnly
        onlyLS.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\onlyLS.csv', index=False, header=True) # saving file to disk
        capMex = capMex.append(onlyLS)                                  # append total output of Lansing data
        lsCount = len(onlyLS.index)
        
    else:        #"Kalamazoo"
        onlyKZ = mexOnly
        onlyKZ.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\onlyKZ.csv', index=False, header=True) # saving file to disk
        capMex = capMex.append(onlyKZ)                                  # append total output of Kalamazoo data
        kzCount = len(onlyLS.index)
        capMex.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\capMex.csv', index=False, header=True) # saving capMex file to disk

print('Detroit header(5), Total count: ', dtCount)
onlyDT.head()
print('Grand Rapids header(5), Total count: ', grCount)
onlyGR.head()
print('Lansing header(5), Total count: ', lsCount)
onlyLS.head()
print('Kalamazoo header(5), Total count: ', kzCount)
onlyKZ.head()



Detroit header(5), Total count:  28


,id,name,city
0,4a46bd39f964a5205fa91fe3,Mexican Town Restaurant,Detroit
1,4ad36269f964a52012e420e3,Mexican Village Restaurant,Detroit
2,4b50da97f964a520663527e3,Qdoba Mexican Grill,Detroit
3,4c615cb7de6920a1a5169864,La Salsa Mexican Taqueria,Detroit
4,4b104dcaf964a520656d23e3,On The Border Mexican Grill & Cantina,Detroit


Grand Rapids header(5), Total count:  29


,id,name,city
0,4e6e28afae60394c6115cd31,Qdoba Mexican Grill,Grand Rapids
1,5127990ae4b0111e6daceb0c,Donk's Mexican Joint,Grand Rapids
2,4b47db96f964a520cd4126e3,On The Border Mexican Grill & Cantina,Grand Rapids
3,4b639c54f964a520da862ae3,Cantina Mexican Grill,Grand Rapids
4,4b7e3060f964a5208ce52fe3,Taquitos Mexican Grill,Grand Rapids


Lansing header(5), Total count:  23


,id,name,city
0,4b686ebcf964a52049782be3,Pablo's Old Town Mexican Restaurant,Lansing
1,4b5c6ee7f964a5209a2f29e3,Pancheros Mexican Grill,Lansing
2,4b7d7d75f964a520eac12fe3,Cancun Mexican Grill,Lansing
3,4b2909f4f964a520a29724e3,Qdoba Mexican Grill,Lansing
4,4b5f67fef964a52084b829e3,Chipotle Mexican Grill,Lansing


Kalamazoo header(5), Total count:  23


,id,name,city
0,4ce147cb0de52c0f5e8ca10c,Juanito's Mexican Restaurant,Kalamazoo
1,4b6090b1f964a520f6ed29e3,QDOBA Mexican Eats,Kalamazoo
2,4ddac39ce4cd1a49b7cefb8a,Qdoba Mexican Grill,Kalamazoo
3,5882ce2654073d0c8d72e558,Qdoba Mexican Eats,Kalamazoo
4,4b4b691af964a520919a26e3,Qdoba Mexican Grill,Kalamazoo
